In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import BayesianRidge, LinearRegression
#import matplotlib.pyplot as plt
import os
import matplotlib as mpl
mpl.use('Qt4Agg')
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Qt4Agg')
import matplotlib.pyplot as plt

In [2]:
# Load Data
data = pd.read_csv('Key_indicator_districtwise.csv')
features = data.columns
y_labels = ['YY_Infant_Mortality_Rate_Imr_Total_Person']
rem_labels = ['YY_Infant_Mortality_Rate_Imr_Total_Person',
              'YY_Post_Neo_Natal_Mortality_Rate_Total',
            'YY_Neo_Natal_Mortality_Rate_Total',
            'YY_Infant_Mortality_Rate_Imr_Total_Female',
        'ZZ_Infant_Mortality_Rate_Total_Lower_Limit',
        'ZZ_Infant_Mortality_Rate_Total_Upper_Limit',
        'YY_Infant_Mortality_Rate_Imr_Total_Male',
        'YY_Post_Neo_Natal_Mortality_Rate_Rural',
       'ZZ_Under_Five_Mortality_Rate_U5MR_Rural_Lower_Limit',
       'YY_Under_Five_Mortality_Rate_U5MR_Rural_Person',
       'YY_Under_Five_Mortality_Rate_U5MR_Rural_Male',
       'YY_Under_Five_Mortality_Rate_U5MR_Rural_Female',
       'ZZ_Under_Five_Mortality_Rate_U5MR_Rural_Upper_Limit',
       'YY_Under_Five_Mortality_Rate_U5MR_Total_Male',
       'YY_Under_Five_Mortality_Rate_U5MR_Total_Person',
       'ZZ_Under_Five_Mortality_Rate_U5MR_Total_Lower_Limit',
       'ZZ_Under_Five_Mortality_Rate_U5MR_Total_Upper_Limit',
       'YY_Under_Five_Mortality_Rate_U5MR_Total_Female',
       'YY_Neo_Natal_Mortality_Rate_Rural',
       'YY_Infant_Mortality_Rate_Imr_Rural_Female',
       'ZZ_Infant_Mortality_Rate_Rural_Lower_Limit',
       'ZZ_Infant_Mortality_Rate_Rural_Upper_Limit',
       'YY_Infant_Mortality_Rate_Imr_Rural_Person',
       'YY_Infant_Mortality_Rate_Imr_Rural_Male']
rem_indices = np.isin(features, rem_labels)
y = data[y_labels].values
x_raw = data.values
x_raw2 = x_raw[:,~rem_indices]
x_raw_labels = features[~rem_indices]
x1 = x_raw2.copy()
x_raw_labels1 = x_raw_labels.copy()

for i in range(17,0,-3):
    x1[:, i+1] = x1[:, i+1]/x1[:, i]
    x1[:, i+2] = x1[:, i+2]/x1[:, i]
    x1 = np.delete(x1, obj=i, axis=1)
    x_raw_labels1 = np.delete(x_raw_labels1, obj=1)

In [3]:
print(np.__version__)

1.13.3


In [4]:
# make state and district categorical variables
states = pd.get_dummies(x_raw[:,0])
dists = pd.get_dummies(x_raw[:,1])
state_cols = states.columns
dists_cols = dists.columns
new_cols = np.append(state_cols, dists_cols)
new_vars = np.append(states, dists, axis=1)
# then append
x1 = np.delete(x1, obj=1, axis=1)
x1 = np.delete(x1, obj=0, axis=1)
x_raw_labels1 = np.delete(x_raw_labels1, obj=1)
x_raw_labels1 = np.delete(x_raw_labels1, obj=0)

#x_raw_labels1 = np.append(new_cols, x_raw_labels1)
#x_new = np.append(new_vars, x1, axis=1)
#x = x_new
#x_labels = x_raw_labels1
x = x1
x_labels = x_raw_labels1

In [5]:
# Remove NaNs (impute)
z = np.zeros(x.shape[1])
for i in range(0,x.shape[0]):
    for j in range(0,x.shape[1]):
        z[j] += np.isnan(x[i,j])

In [6]:
print(z.shape)
print(x.shape)
print(x_labels.shape)
x_no_nans = x[:,z == 0]
x_no_nans_labels = x_labels[z == 0]

(612,)
(284, 612)
(612,)


In [7]:
x_pandas = pd.DataFrame(x, columns=x_labels)
x_pandas['YY_Infant_Mortality_Rate_Imr_Total_Person']=y
x_pandas.to_csv("Clean_Key_indicator_districtwise.csv")

In [8]:
x_pandas = pd.DataFrame(x, columns=x_labels)
correlations = x_pandas.astype(float).corr()
fig1 = plt.figure()
ax = fig1.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig1.colorbar(cax)
plt.show()
fig2 = plt.figure()
ax = fig2.add_subplot(111)
cax = ax.matshow(correlations.iloc[250:350, 250:350], vmin=-1, vmax=1)
fig2.colorbar(cax)
plt.show()
fig3 = plt.figure()
ax = fig3.add_subplot(111)
cax = ax.matshow(correlations.iloc[290:310, 290:310], vmin=-1, vmax=1)
fig3.colorbar(cax)
plt.show()


In [ ]:
x_pandas

In [9]:
#a = correlations.iloc[850:, 850:]
#correlations[correlations>0.8]
#fig1 = plt.figure()
#ax = fig1.add_subplot(111)
#cax = ax.matshow(a, vmin=-1, vmax=1)
#fig.colorbar(cax)
#plt.show()

In [10]:
x_no_nans[:,690:]
x_no_nans_labels[690:]

Index([], dtype='object')

In [193]:
x_pandas_nonan = pd.DataFrame(x_no_nans, columns=x_no_nans_labels)
correlations_nonan = x_pandas_nonan.astype(float).corr()
fig2 = plt.figure()



In [12]:
ax = fig2.add_subplot(111)
cax = ax.matshow(correlations_nonan, vmin=-1, vmax=1)
fig2.colorbar(cax)
plt.show()
fig3 = plt.figure()

In [13]:
ax = fig3.add_subplot(111)
cax = ax.matshow(correlations_nonan.iloc[290:300,290:300], vmin=-1, vmax=1)
fig3.colorbar(cax)
plt.show()

In [14]:
fig1.savefig('correlations.png',  dpi = 1000)
fig2.savefig('correlations_nonan.png', dpi = 1000)

KeyboardInterrupt: 

In [15]:
correlations_nonan.iloc[0:30,0:30]

,AA_Population_Total,AA_Population_Rural,AA_Population_Urban,AA_Ever_Married_Women_Aged_15_49_Years_Total,AA_Ever_Married_Women_Aged_15_49_Years_Rural,AA_Ever_Married_Women_Aged_15_49_Years_Urban,AA_Currently_Married_Women_Aged_15_49_Years_Total,AA_Currently_Married_Women_Aged_15_49_Years_Rural,AA_Currently_Married_Women_Aged_15_49_Years_Urban,AA_Children_12_23_Months_Total,...,BB_Population_Below_Age_15_Years_Total,BB_Population_Below_Age_15_Years_Rural,BB_Population_Below_Age_15_Years_Urban,BB_Dependency_Ratio_Total,BB_Dependency_Ratio_Rural,BB_Dependency_Ratio_Urban,BB_Currently_Married_Illiterate_Women_Aged_15_49_Years_Total,BB_Currently_Married_Illiterate_Women_Aged_15_49_Years_Rural,BB_Currently_Married_Illiterate_Women_Aged_15_49_Years_Urban,CC_Sex_Ratio_At_Birth_Total
AA_Population_Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Population_Rural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Population_Urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Ever_Married_Women_Aged_15_49_Years_Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Ever_Married_Women_Aged_15_49_Years_Rural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Ever_Married_Women_Aged_15_49_Years_Urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA_Currently_Married_Women_Aged_15_49_Years_Total,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,-1.000000,0.999930,-0.999930,...,0.462233,0.280144,0.264534,0.550807,0.369753,0.311441,0.413994,0.178023,0.187900,0.207589
AA_Currently_Married_Women_Aged_15_49_Years_Rural,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,1.000000,-0.999930,0.999930,...,-0.462233,-0.280144,-0.264534,-0.550807,-0.369753,-0.311441,-0.413994,-0.178023,-0.187900,-0.207589
AA_Currently_Married_Women_Aged_15_49_Years_Urban,NaN,NaN,NaN,NaN,NaN,NaN,0.999930,-0.999930,1.000000,-1.000000,...,0.463187,0.281349,0.265361,0.551341,0.370547,0.312085,0.415421,0.179566,0.189216,0.209186
AA_Children_12_23_Months_Total,NaN,NaN,NaN,NaN,NaN,NaN,-0.999930,0.999930,-1.000000,1.000000,...,-0.463187,-0.281349,-0.265361,-0.551341,-0.370547,-0.312085,-0.415421,-0.179566,-0.189216,-0.209186


In [194]:
fig4 = plt.figure()
ax = fig4.add_subplot(111)
cax = ax.matshow(correlations_nonan.iloc)
fig4.colorbar(cax)
plt.show()

ValueError: need more than 0 values to unpack

In [195]:
split = int(x_no_nans.shape[0]*0.9)
x_no_nans_train = x_no_nans[:split]
x_no_nans_test = x_no_nans[split:]
y_train = y[:split]
y_test = y[split:]

In [196]:
clf = BayesianRidge(compute_score=True, normalize=True)
clf.fit(x_no_nans_train, y_train)

ols = LinearRegression(normalize=True)
ols.fit(x_no_nans_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [197]:
bayesian_coefs = clf.coef_
sorted_coefs = np.sort(np.abs(bayesian_coefs))
sorted_indices = np.argsort(np.abs(bayesian_coefs))
sorted_labels = x_no_nans_labels[sorted_indices]
sorted_labels[-100:]
for el in sorted_labels[-10:]:
    print el

ZZ_Crude_Death_Rate_Rural_Upper_Limit
YY_Crude_Death_Rate_Cdr_Rural_Female
YY_Crude_Death_Rate_Cdr_Total_Female
NN_Current_Usage_Emergency_Contraceptive_Pills_Rural
LL_Mean_Number_Of_Children_Surviving_To_Women_Aged_15_49_Years_Rural
MM_Average_Month_Of_Pregnancy_At_The_Time_Of_Abortion_Total
AA_Currently_Married_Women_Aged_15_49_Years_Rural
AA_Currently_Married_Women_Aged_15_49_Years_Total
AA_Children_12_23_Months_Total
AA_Currently_Married_Women_Aged_15_49_Years_Urban


In [192]:
least_sq = ols.coef_
ols_sorted_coefs = np.sort(np.abs(least_sq))
ols_sorted_indices = np.argsort(np.abs(least_sq))
ols_sorted_labels = x_no_nans_labels[ols_sorted_indices]
ols_sorted_labels[0,-100:]
for el in ols_sorted_labels[0,-10:]:
    print el

ZZ_Crude_Death_Rate_Rural_Upper_Limit
VV_Average_Month_By_Which_Children_Received_Foods_Other_Than_Breast_Milk_Semi_Solid_Mashed_Food_Rural
LL_Median_Age_At_First_Live_Birth_Of_Women_Aged_15_49_Years_Rural
BB_Average_Household_Size_Sc_Total
NN_Current_Usage_Emergency_Contraceptive_Pills_Rural
AA_Children_12_23_Months_Total
AA_Currently_Married_Women_Aged_15_49_Years_Urban
NN_Current_Usage_Emergency_Contraceptive_Pills_Total
LL_Mean_Number_Of_Children_Surviving_To_Women_Aged_15_49_Years_Rural
LL_Mean_Number_Of_Children_Surviving_To_Women_Aged_15_49_Years_Total


In [191]:
print(len(set(list(ols_sorted_labels[0,-100:])+list(sorted_labels[-100:]))))
print(len(np.intersect1d(ols_sorted_labels[0,-20:], sorted_labels[-10:])))
print(len(np.intersect1d(ols_sorted_labels[0,-10:], sorted_labels[-20:])))
print(len(np.intersect1d(ols_sorted_labels[0,-10:], sorted_labels[-10:])))
np.intersect1d(ols_sorted_labels[0,-10:], sorted_labels[-10:])

129
5
5
5


array(['AA_Children_12_23_Months_Total',
       'AA_Currently_Married_Women_Aged_15_49_Years_Urban',
       'LL_Mean_Number_Of_Children_Surviving_To_Women_Aged_15_49_Years_Rural',
       'NN_Current_Usage_Emergency_Contraceptive_Pills_Rural',
       'ZZ_Crude_Death_Rate_Rural_Upper_Limit'], dtype=object)

In [199]:
ols_yhat = ols.predict(x_no_nans_test)
clf_yhat = clf.predict(x_no_nans_test)

print(ols.score(x_no_nans_test, y_test))
print(clf.score(x_no_nans_test, y_test))

print(ols.score(x_no_nans_train, y_train))
print(clf.score(x_no_nans_train, y_train))


-0.135157057076
0.670084344816
1.0
0.792663816516


In [23]:
print(x_no_nans_train.shape)

(255, 413)


In [261]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 180)
PCA_train_X = pca.fit_transform(x_no_nans_train)


In [262]:
print(sum(pca.explained_variance_ratio_))
#print(pca.components_)
print(PCA_train_X.shape)

0.999999950857
(255, 180)


In [263]:
clf_PCA = BayesianRidge(compute_score=True, normalize=True)
clf_PCA.fit(PCA_train_X, y_train)

ols_PCA = LinearRegression(normalize=True)
ols_PCA.fit(PCA_train_X, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [264]:
PCA_test_X = pca.transform(x_no_nans_test)
print(PCA_test_X.shape)

(29, 180)


In [265]:
print(clf.coef_.shape)
print(PCA_test_X.shape)
print(PCA_train_X.shape)
print(y_test.shape)
print(y_train.shape)

(413,)
(29, 180)
(255, 180)
(29, 1)
(255, 1)


In [266]:
ols_PCA_yhat = ols_PCA.predict(PCA_test_X )
clf_PCA_yhat = clf_PCA.predict(PCA_test_X )

print(ols_PCA.score(PCA_test_X, y_test))
print(clf_PCA.score(PCA_test_X, y_test))

print(ols_PCA.score(PCA_train_X, y_train))
print(clf_PCA.score(PCA_train_X, y_train))

0.683851796782
0.698742902525
0.893078618114
0.819345272348
